In [1]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stat
import math
import scipy.sparse as sp

import numpy_groupies as npg
import multiprocessing
sys.path.append("/Users/lmartin/Documents/2023/CDK4_mutsig_added_pats/CurveBall/cbsig")
import curveball

In [2]:
combined_maf = pd.read_csv('/Users/lmartin/Documents/2025/RA_combined_maf_dedup_updated_on_trees_022025.maf',sep='\t')
subclones_sig_genes = pd.read_csv('/Users/lmartin/Documents/2023/CDK4_mutsig_added_pats/on_subclones_v3/0/mutsig_results/outdir/sig_genes.txt',sep='\t')



In [3]:
subset=(subclones_sig_genes['p'] < 0.001)

In [4]:
coding_muts=['Missense_Mutation',
 'Nonsense_Mutation',
 'Start_Codon_SNP',
 'De_novo_Start_InFrame',
 'De_novo_Start_OutOfFrame',
 'Nonstop_Mutation',
 'Frame_Shift_Del',
 'Frame_Shift_Ins',
 'In_Frame_Del',
 'In_Frame_Ins']

coding_maf=combined_maf[combined_maf['Variant_Classification'].isin(coding_muts)]
len(coding_maf['Hugo_Symbol'].unique())

6790

In [5]:
pd.concat([
  coding_maf.groupby(["Hugo_Symbol", "Patient_ID"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_pats"),
  coding_maf.groupby(["Hugo_Symbol", "Tumor_Sample_Barcode"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_clones")],
  axis = 1
)

,n_mut_pats,n_mut_clones
Hugo_Symbol,,
CSAD,2,3
CTCF,2,4
CTNNA2,5,5
ESR1,6,10
HIST1H3B,2,3
IVL,4,6
KMT2C,5,16
KRAS,3,3
MEN1,1,2


In [6]:
coding_maf.sort_values(by='Tumor_Sample_Barcode')['Tumor_Sample_Barcode'].unique()

array(['1002_CL1', '1002_CL10', '1002_CL11', '1002_CL12', '1002_CL13',
       '1002_CL14', '1002_CL15', '1002_CL16', '1002_CL17', '1002_CL18',
       '1002_CL19', '1002_CL20', '1002_CL21', '1002_CL3', '1002_CL5',
       '1002_CL6', '1002_CL8', '1035_CL1', '1035_CL10', '1035_CL2',
       '1035_CL3', '1035_CL4', '1035_CL5', '1035_CL6', '1035_CL7',
       '1035_CL8', '1035_CL9', '1045_CL1', '1045_CL10', '1045_CL11',
       '1045_CL12', '1045_CL13', '1045_CL14', '1045_CL15', '1045_CL16',
       '1045_CL3', '1045_CL4', '1045_CL6', '1045_CL7', '1045_CL9',
       '1078_CL1', '1078_CL2', '1078_CL4', '1078_CL5', '1078_CL6',
       '1113_CL1', '1113_CL10', '1113_CL11', '1113_CL12', '1113_CL13',
       '1113_CL2', '1113_CL3', '1113_CL5', '1113_CL8', '1326_CL1',
       '1326_CL10', '1326_CL12', '1326_CL13', '1326_CL2', '1326_CL3',
       '1326_CL5', '1326_CL6', '1326_CL7', '1326_CL8', '1326_CL9',
       '1534_CL1', '1534_CL2', '1534_CL3', '1534_CL4', '1547_CL1',
       '1547_CL11', '1547_CL12', '1

In [7]:
## sort coding maf
coding_maf = coding_maf.sort_values(by=['Tumor_Sample_Barcode'])
coding_maf = coding_maf.reset_index()
coding_maf.drop(columns=['index'],inplace=True)

In [8]:
coding_maf

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high,Tumor_Sample_Barcode
0,1002,pre,NaN,TLX2,2,74743128,G,C,10,6,...,1,NaN,NaN,0.81,0.59,1.00,1.00,0.99,1.00,1002_CL1
1,1002,pre,NaN,SEC31B,10,102267765,G,C,90,38,...,1,NaN,NaN,0.88,0.75,1.00,1.00,0.99,1.00,1002_CL1
2,1002,pre,NaN,OR6K6,1,158725164,G,C,95,38,...,1,NaN,NaN,0.87,0.73,1.00,1.00,0.99,1.00,1002_CL1
3,1002,pre,NaN,FAXC,6,99729273,C,G,64,28,...,1,NaN,NaN,0.91,0.80,1.00,1.00,0.99,1.00,1002_CL1
4,1002,pre,NaN,BAI2,1,32196555,C,T,9,2,...,1,NaN,NaN,0.60,0.28,1.00,1.00,0.99,1.00,1002_CL1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11099,2974,7,NaN,RAVER2,1,65273093,G,A,285,7,...,9,NaN,NaN,0.09,0.04,0.15,0.01,0.01,0.01,2974_CL9
11100,2974,cfDNA_1976,NaN,ZNF93,19,20045112,A,G,113,4,...,9,NaN,NaN,0.19,0.06,0.32,0.04,0.04,0.04,2974_CL9
11101,2974,cfDNA_1976,NaN,ZNF737,19,20728105,C,T,259,10,...,9,NaN,NaN,0.18,0.08,0.28,0.04,0.04,0.04,2974_CL9
11102,2974,cfDNA_1976,NaN,ZNF729,19,22498344,G,A,145,3,...,9,NaN,NaN,0.12,0.03,0.22,0.04,0.04,0.04,2974_CL9


## test with KMT2C

In [9]:
KMT2C_all_muts = coding_maf[coding_maf['Hugo_Symbol']=='KMT2C']
KMT2C_all_muts

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high,Tumor_Sample_Barcode
4112,1326,RP-1700_BWES00001_v1_Exome_OnPrem,NaN,KMT2C,7,151896451,T,A,36,13,...,2,NaN,NaN,0.80,0.57,1.00,0.98,0.96,1.00,1326_CL2
4145,1326,RP-1700_T01027-3_v4_Exome_OnPrem,NaN,KMT2C,7,151945382,C,-,246,64,...,9,NaN,NaN,0.87,0.74,1.00,0.83,0.77,0.89,1326_CL9
4588,1547,10,NaN,KMT2C,7,151879260,C,A,184,12,...,14,NaN,NaN,0.18,0.10,0.27,0.06,0.06,0.06,1547_CL14
4650,1547,6,NaN,KMT2C,7,151874772,C,T,180,6,...,15,NaN,NaN,0.09,0.04,0.14,0.05,0.05,0.06,1547_CL15
4724,1558,RA_1558_cfDNA_3,NaN,KMT2C,7,151879322,G,A,79,33,...,1,NaN,NaN,0.89,0.76,1.00,1.00,0.99,1.00,1558_CL1
4780,1558,1558_6,NaN,KMT2C,7,152012253,G,C,185,21,...,10,NaN,NaN,0.39,0.25,0.53,0.30,0.29,0.30,1558_CL10
5466,2389,4,NaN,KMT2C,7,151878626,C,A,122,5,...,12,NaN,NaN,0.16,0.06,0.25,0.36,0.35,0.37,2389_CL12
6202,2389,8,NaN,KMT2C,7,152007146,G,A,211,1,...,17,NaN,NaN,0.34,0.01,0.78,0.01,0.01,0.01,2389_CL17
6379,2389,7,NaN,KMT2C,7,151878044,G,A,194,1,...,18,NaN,NaN,0.05,0.01,0.09,0.01,0.01,0.01,2389_CL18
6487,2389,11,NaN,KMT2C,7,151874215,C,A,61,1,...,18,NaN,NaN,0.06,0.01,0.10,0.01,0.01,0.01,2389_CL18


In [10]:
## total number of mutations in the target gene N0
N0 = len(KMT2C_all_muts)
## total number of clones
N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )

In [11]:
N0

21

In [12]:
N_c

159

## start with one patient

In [10]:
RA_2389 = coding_maf[coding_maf['Patient_ID']==2389]

### for every clone, I am going to model the probability that KMT2C is in the clone or not


In [58]:

RA_2389_clones = list(RA_2389['Tumor_Sample_Barcode'].unique() )
# total number of mutations in this patient
M = len(RA_2389)
 
# initialize a Pij matrix where you have a 1 if a clone was mutated and then the score
P_2389 =np.zeros( (len(RA_2389_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_2389_clones:
    clone_df = RA_2389[RA_2389['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    
    # a given clone has no mutation in the gene
    P_2389[clone_idx, 0] =  term_to_calc
    
    # a given clone has one or more mutations in the gene
    P_2389[clone_idx, 1] = 1- term_to_calc
    

    clone_idx +=1


In [59]:
## now every row sums to one

In [60]:
P_2389_prob_df = pd.DataFrame(P_2389, columns=['no_KMT2C', 'KMT2C'])
P_2389_prob_df.insert(0, 'clone', RA_2389_clones)
P_2389_prob_df

,clone,no_KMT2C,KMT2C
0,2389_CL16,0.862730,0.137270
1,2389_CL5,0.938615,0.061385
2,2389_CL3,0.927032,0.072968
3,2389_CL18,0.359055,0.640945
4,2389_CL14,0.916851,0.083149
5,2389_CL15,0.758178,0.241822
6,2389_CL1,0.979575,0.020425
7,2389_CL17,0.834506,0.165494
8,2389_CL6,0.920656,0.079344
9,2389_CL7,0.939910,0.060090


In [64]:
RA_2389.groupby('Tumor_Sample_Barcode').count()

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Variant_Type,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high
Tumor_Sample_Barcode,,,,,,,,,,,,,,,,,,,,,
2389_CL1,17,17,0,17,17,17,17,17,17,17,...,17,17,0,0,17,17,17,17,17,17
2389_CL10,22,22,0,22,22,22,22,22,22,22,...,22,22,0,0,22,22,22,22,22,22
2389_CL11,65,65,0,65,65,65,65,65,65,65,...,65,65,0,0,65,65,65,65,65,65
2389_CL12,87,87,0,87,87,87,87,87,87,87,...,87,87,0,0,87,87,87,87,87,87
2389_CL13,206,206,0,206,206,206,206,206,206,206,...,206,206,0,0,206,206,206,206,206,206
2389_CL14,64,64,0,64,64,64,64,64,64,64,...,64,64,0,0,64,64,64,64,64,64
2389_CL15,205,205,0,205,205,205,205,205,205,205,...,205,205,0,0,205,205,205,205,205,205
2389_CL16,112,112,0,112,112,112,112,112,112,112,...,112,112,0,0,112,112,112,112,112,112
2389_CL17,133,133,0,133,133,133,133,133,133,133,...,133,133,0,0,133,133,133,133,133,133


## patient score if no KMT2C mutation in any clone

In [13]:
P_2389_prob_df_no_KMT2C = P_2389_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C = [ np.log(p) for p in P_2389_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C))

2.6946982684320844


## patient score if a KMT2C mutation in all clones

In [14]:
P_2389_prob_df_KMT2C = P_2389_prob_df['KMT2C'].to_list()
logs_KMT2C = [ np.log(p) for p in P_2389_prob_df_KMT2C]
print( -np.sum(logs_KMT2C))

37.92316367280446


## test RA 1598, no KMT2C

In [15]:
RA_1598= coding_maf[coding_maf['Patient_ID']==1598]
RA_1598_clones = list(RA_1598['Tumor_Sample_Barcode'].unique() )

# total number of mutations in this patient
M = len(RA_1598)
# initialize a Pij matrix where you have a 1 if a clone was mutated 
P_1598 =np.zeros( (len(RA_1598_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_1598_clones:
    clone_df = RA_1598[RA_1598['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    # a given clone has no mutation in the gene
    P_1598[clone_idx, 0] =  term_to_calc
    
    #  a given clone has a mutation in the gene
    P_1598[clone_idx, 1] = 1-term_to_calc
    

    clone_idx +=1

In [16]:
P_1598_prob_df = pd.DataFrame(P_1598, columns=['no_KMT2C', 'KMT2C'])
P_1598_prob_df.insert(0, 'clone', RA_1598_clones)
P_1598_prob_df

,clone,no_KMT2C,KMT2C
0,1598_CL2,0.931663,0.068337
1,1598_CL1,0.867789,0.132211
2,1598_CL8,0.700242,0.299758
3,1598_CL10,0.269250,0.730750
4,1598_CL5,0.883355,0.116645
5,1598_CL4,0.931663,0.068337
6,1598_CL7,0.793799,0.206201
7,1598_CL9,0.867789,0.132211
8,1598_CL3,0.899186,0.100814
9,1598_CL6,0.793799,0.206201


## patient score if no KMT2C clones (truth)

In [17]:
P_1598_prob_df_no_KMT2C = P_1598_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1598))

2.785767361195695


## patient score if all KMT2C clones

In [18]:
P_1598_prob_df_KMT2C = P_1598_prob_df['KMT2C'].to_list()
logs_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_KMT2C]
print( -np.sum(logs_KMT2C_1598))

18.532696469275578


## try for all patients for KMT2C

In [427]:
all_probs_KMT2C = []

for patient in all_patients: 

    RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
    RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )

    ## total number of mutations in the target gene N0
    N0 = len(KMT2C_all_muts)
    ## total number of clones
    N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )

    # total number of mutations in this patient
    M = len(RA_maf)
    # initialize a Pij matrix where you have a 1 if a clone was mutated 
    P_pat =np.zeros( (len(RA_maf_clones), 2 ))
    
    gene="KMT2C"
    
    clone_idx=0
    for clone in RA_maf_clones:
        clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
        mutations_in_clone = clone_df['Hugo_Symbol'].unique()
        Mij = len(list(mutations_in_clone))
        
        term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
        
        # probability that a given clone has no mutation in the gene
        P_pat[clone_idx, 0] = term_to_calc
        
        # probability that a given clone has one or more mutations in the gene
        P_pat[clone_idx, 1] = 1- term_to_calc
        clone_idx +=1

    P_prob_df = pd.DataFrame(P_pat, columns=['no_KMT2C', 'KMT2C'])
    P_prob_df.insert(0, 'clone', RA_maf_clones)
    all_probs_KMT2C.append(P_prob_df)
    

## score if no KMT2C mutation in this patient

In [53]:
probs_1326_no_KMT2C = probs_1326['no_KMT2C'].to_list()
logs_no_KMT2C_1326 = [ np.log(p) for p in probs_1326_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1326))

2.785239956066282


## real distribution

In [54]:
KMT2C_clones_RA1326= ['1326_CL2','1326_CL9']

In [55]:
P_1326_prob_df_KMT2C_clones = probs_1326[probs_1326['clone'].isin(KMT2C_clones_RA1326)]
P_1326_prob_df_non_KMT2C_clones = probs_1326[~probs_1326['clone'].isin(KMT2C_clones_RA1326)]

P_1326_prob_df_KMT2C_clones_list = P_1326_prob_df_KMT2C_clones['KMT2C'].to_list()
logs_real_KMT2C = [ -np.log(p) for p in P_1326_prob_df_KMT2C_clones_list]

P_1326_no_KMT2C_clones_list = P_1326_prob_df_non_KMT2C_clones['no_KMT2C'].to_list()
logs_real_no_KMT2C = [ -np.log(p) for p in P_1326_no_KMT2C_clones_list]


np.sum( np.concatenate((logs_real_KMT2C, logs_real_no_KMT2C)))

4.547794414031647

In [29]:
pats_with_no_KMT2C = [1598, 1078, 1045, 1113, 1644, 2819, 2974, 1035,1534,1002]
pats_with_no_KMT2C_idx =[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [44]:
probs_pats_no_KMT2C = []
probs_pats_no_KMT2C_scores = []
probs_pats_no_KMT2C_scores_dict = {}
for idx in pats_with_no_KMT2C_idx:
    score = 0
    probs_pats_no_KMT2C.append(all_probs_KMT2C[idx])
    pat_probs = all_probs_KMT2C[idx]
    pat_probs_no_KMT2C = pat_probs['no_KMT2C'].to_list()
    logs_no_KMT2C = [ np.log(p) for p in pat_probs_no_KMT2C]
    score = -np.sum(logs_no_KMT2C)
    
    probs_pats_no_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_no_KMT2C_scores_dict[pat] = score


In [46]:
pats_with_KMT2C_idx=[0, 10,11,12,13]

In [47]:
all_KMT2C_clones = list(KMT2C_all_muts['Tumor_Sample_Barcode'].unique())

In [56]:

probs_pats_with_KMT2C = []
probs_pats_KMT2C_scores = []
probs_pats_with_KMT2C_scores_dict = {}
for idx in pats_with_KMT2C_idx:
    #score=0
    running_score=[]
    pat_probs = all_probs_KMT2C[idx]
    all_clones_in_pat = list(pat_probs['clone'])
    for clone in all_clones_in_pat:
        pat_probs_clone = pat_probs[pat_probs['clone']==clone]
        if clone in all_KMT2C_clones:

            pat_probs_clone_KMT2C = pat_probs_clone['KMT2C'].to_list()
            logs_KMT2C = [ - np.log(p) for p in pat_probs_clone_KMT2C]
            running_score.extend(logs_KMT2C)
        else:
            pat_probs_no_KMT2C = pat_probs_clone['no_KMT2C'].to_list()
            logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
            running_score.extend(logs_no_KMT2C)
            
    score = np.sum(running_score)
    
    probs_pats_with_KMT2C.append(pat_probs)
    probs_pats_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_with_KMT2C_scores_dict[pat] = score


In [57]:
probs_pats_with_KMT2C_scores_dict

{1547: 2.8031613827454733,
 1558: 3.403834833843852,
 1326: 4.547794414031647,
 2389: 14.714030365894862,
 2542: 8.820179860729803}

## test any gene

In [54]:
def calculate_distributions(gene):
    all_patients = list(coding_maf['Patient_ID'].unique())
    gene_all_muts = coding_maf[coding_maf['Hugo_Symbol']==gene]
    all_probs = []
    
    for patient in all_patients: 
    
        RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
        RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )
        
        ## total number of mutations N0
        N0 = len(gene_all_muts)
        ## total number of mutated clones N_c
        N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )
        
        # total number of mutations in this patient
        M = len(RA_maf)
        # initialize a Pij matrix where you have a 1 if a clone was mutated 
        P_pat =np.zeros( (len(RA_maf_clones), 2 ))
        
        clone_idx=0
        for clone in RA_maf_clones:
            clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
            mutations_in_clone = clone_df['Hugo_Symbol'].unique()
            Mij = len(list(mutations_in_clone))
            term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0

            # probability that a given clone has no mutation in the gene
            P_pat[clone_idx, 0] = term_to_calc
            
            
            # probability that a given clone has a mutation in the gene
            P_pat[clone_idx, 1] = 1-term_to_calc
            clone_idx +=1

    
        P_prob_df = pd.DataFrame(P_pat, columns=[0, 1])
        P_prob_df.insert(0, 'clone', RA_maf_clones)
        all_probs.append(P_prob_df)
    
    all_probs_concat = pd.concat(all_probs)
    pats_label=[]
    clone_labels = list(all_probs_concat['clone'])
    for clone in clone_labels:
        pat = clone.split('_')[0]
        pats_label.append(pat)

    all_probs_df_labeled=all_probs_concat.copy()
    all_probs_df_labeled['patient']=pats_label
    all_probs_df_labeled=all_probs_df_labeled.sort_values(by=['patient'])
    all_probs_df_labeled.reset_index(inplace=True)
    all_probs_df_labeled.drop(columns=['index'],inplace=True)
    return all_probs_df_labeled

In [14]:
esr1_Pm = calculate_distributions('ESR1')

In [55]:
kmt2c_Pm = calculate_distributions('KMT2C')

In [56]:
kmt2c_Pm

,clone,0,1,patient
0,1002_CL1,0.961869,0.038131,1002
1,1002_CL6,0.995255,0.004745,1002
2,1002_CL5,0.992890,0.007110,1002
3,1002_CL3,0.987393,0.012607,1002
4,1002_CL21,0.203368,0.796632,1002
...,...,...,...,...
154,2974_CL3,0.955739,0.044261,2974
155,2974_CL4,0.913348,0.086652,2974
156,2974_CL5,0.833877,0.166123,2974
157,2974_CL6,0.955739,0.044261,2974


In [17]:
np.unique(KMT2C_all_muts['Tumor_Sample_Barcode'])

array(['1326_CL2', '1326_CL9', '1547_CL14', '1547_CL15', '1558_CL1',
       '1558_CL10', '2389_CL12', '2389_CL17', '2389_CL18', '2389_CL4',
       '2389_CL5', '2389_CL7', '2542_CL12', '2542_CL15', '2542_CL18',
       '2542_CL2'], dtype=object)

In [18]:
pm_clones, pm_c_ui, pm_c_idx = np.unique(kmt2c_Pm["clone"], return_inverse = True, return_index = True)

sorted_pats, pm_pat_idx, total_clone_per_pat = np.unique(kmt2c_Pm["patient"], return_inverse = True, return_counts=True)
pat_to_clone_idx = pm_pat_idx[pm_c_ui]



In [19]:
sorted_pats

array(['1002', '1035', '1045', '1078', '1113', '1326', '1534', '1547',
       '1558', '1598', '1644', '2389', '2542', '2819', '2974'],
      dtype=object)

In [20]:
mut_patients = np.unique(KMT2C_all_muts['Patient_ID'])
mut_patients

array([1326, 1547, 1558, 2389, 2542])

## permutations

In [ ]:
## so I am using the curveball to continue to permute the number of mutations per clone given the number of mutations and clones in the data

## OK so my distribution is the probability matrix that I get from the observed
## then through the curveball, I permute # of clones mutated and calculate the score
## then compare the real score to the observed score under the perms

## in this case it is also important to know which clones I am permuting

## so i have the whole distribution and vectorized real scores
## need to compute scores in a better way

In [57]:

S = subclones_sig_genes
"""
_, S["qCV"], _, _ = multitest.multipletests(S["pCV"], method = "fdr_bh")
subset_idx = (S["p"] < 0.001)
"""

subset_idx=(subclones_sig_genes['gene'] =='KMT2C')

In [58]:
clone_order= list(np.sort(coding_maf["Tumor_Sample_Barcode"]))

In [59]:
order_map = {val:i for i,val in enumerate(clone_order)}
kmt2c_Pm['clones_sort']=kmt2c_Pm ['clone'].map(order_map)
kmt2c_Pm = kmt2c_Pm.sort_values(by='clones_sort')
kmt2c_Pm = kmt2c_Pm.drop('clones_sort',axis=1)
kmt2c_Pm.reset_index(inplace=True)
kmt2c_Pm = kmt2c_Pm.drop('index',axis=1)

In [60]:
kmt2c_Pm

,clone,0,1,patient
0,1002_CL1,0.961869,0.038131,1002
1,1002_CL10,0.872157,0.127843,1002
2,1002_CL11,0.991317,0.008683,1002
3,1002_CL12,0.932513,0.067487,1002
4,1002_CL13,0.971077,0.028923,1002
...,...,...,...,...
154,2974_CL4,0.913348,0.086652,2974
155,2974_CL5,0.833877,0.166123,2974
156,2974_CL6,0.955739,0.044261,2974
157,2974_CL8,0.396736,0.603264,2974


In [61]:
maf = coding_maf
## mapping clones to patients
clones, c_ui, c_idx = np.unique(maf["Tumor_Sample_Barcode"], return_inverse = True, return_index = True)

_, p_idx = np.unique(maf["Patient_ID"], return_inverse = True)
c2pat = p_idx[c_ui]

# genes
genes, g_ui, g_idx = np.unique(maf["Hugo_Symbol"], return_inverse = True, return_index = True)
g2gidx = dict(zip(genes, g_idx[g_ui]))

#subset_idx_mat = [g2gidx[x] for x in S.loc[subset_idx, "gene"]]
subset_idx_mat = [g2gidx[x] for x in S.loc[subset_idx, "gene"]]

# CG is matrix size # total of genes x total # of clones
## basically a binarized matrix of the coding maf

# c_idx is the indices of the first occurences of the unique values in the original array
CG = np.array((sp.coo_matrix((np.ones_like(c_idx), (c_idx, g_idx))).todense() > 0))
num_total_mutations = CG.sum(axis=0)
n_genes = len(subset_idx_mat)

In [38]:
len(c_ui)

159

In [26]:
len(g_idx)

11104

In [27]:
len(c_idx)

11104

In [28]:
total_num_mutations_in_gene = num_total_mutations[subset_idx_mat]
total_num_mutations_in_gene

array([16])

In [29]:
c_idx

array([  0,   0,   0, ..., 158, 158, 158])

In [30]:
def compute_mut_clone_indices(pat_idx, mat):
    clone_ids, gene_ids = np.where(mat > 0)  # Get nonzero clone-gene pairs
    return npg.aggregate(pat_idx[clone_ids], list(zip(clone_ids, gene_ids)), func='all', axis=0)

compute_mut_clone_indices(c2pat, CG[:, subset_idx_mat])

array([[False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [ True, False],
       [False, False],
       [ True, False],
       [ True, False],
       [False, False],
       [False, False],
       [ True, False],
       [ True, False]])

In [31]:
## for KMT2C specifically
mut_clones = np.unique(KMT2C_all_muts['Tumor_Sample_Barcode'])

# map clones back to prob dist
mut_clones_in_pm = kmt2c_Pm[kmt2c_Pm["clone"].isin(mut_clones)]
mut_clones_in_pm_idx = mut_clones_in_pm.index.tolist()
mut_clones_in_pm_idx

[54, 64, 73, 79, 89, 90, 106, 107, 109, 110, 112, 117, 122, 130, 133, 136]

In [62]:
def compute_mut_clone_indices(pat_idx, mat):
    mut_clone_indices = {pat: [] for pat in np.unique(pat_idx)}

    for clone_idx, pat in enumerate(pat_idx):
        mutated_clones = np.where(mat[clone_idx, :] > 0)[0]  # Get indices of nonzero mutations

        mut_clone_indices[pat].extend([clone_idx for gene in mutated_clones])  

    return mut_clone_indices

mut_clone_indices_obs = compute_mut_clone_indices(c2pat, CG[:, subset_idx_mat])

In [63]:
mut_clone_indices_obs

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [58, 64],
 6: [],
 7: [73, 74],
 8: [80, 81],
 9: [],
 10: [],
 11: [109, 114, 115, 117, 118, 120],
 12: [125, 128, 131, 133],
 13: [],
 14: []}

In [64]:
unique_pats = list(kmt2c_Pm['patient'].unique())
pat_dict = {val: idx for idx, val in enumerate(unique_pats )}
mapped_values = [pat_dict[val] for val in unique_pats ]

In [131]:
def calculate_scores(mutated_clone_per_pat):
    """
    input: 
    gene_pm: probability distribution of a clone having or not having a mutation in given gene, pre-calculated
    gene_muts_per_pat: vector of len(total patients) with a 0 at pat index if no gene mutated in that patient, 1 if gene mutated in patient
    mut_clones: vector of len(total clones) with a 0 at clone index if no gene mutated in that clone, 1 if gene mutated in clone
    pat_indices: array of len(total patients) that represents # of clones in each patient
    
    returns:
    score_vector: vector of len(total patients) with the scores per patient, computed as a sum over the -log of all clones in the patient,
    -log( 0<=x<=1 ) + -log(x >= 2 ) per patient
    """
    score_vector = []

    for pat in unique_pats:
        pat_probs = kmt2c_Pm[kmt2c_Pm['patient']==pat]
        running_score=[]
        pat_num = pat_dict[pat]
        count_muts = len(mutated_clone_per_pat[pat_num])
        ## we need 2 or more clones to have the mtuation in order to compute the 1 column

        # if 0 <= x <= 1
        if count_muts <= 1: 
            pat_probs_no_KMT2C = pat_probs[0].to_list()
            logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
            running_score.extend(logs_no_KMT2C)

        # if x >= 2
        if count_muts >=2:
            #get indicies of clone
            mut_clones_idx = mutated_clone_per_pat[pat_num]

            all_clones_in_pat_idx = pat_probs.index.tolist()

            for clone_idx in all_clones_in_pat_idx:

                if clone_idx in mut_clones_idx:
                    pat_probs_clone_KMT2C = []
                    pat_probs_test = pat_probs.loc[clone_idx][1]
                    pat_probs_clone_KMT2C.append(pat_probs_test)
                    logs_KMT2C = [ - np.log(p) for p in pat_probs_clone_KMT2C]
                    running_score.extend(logs_KMT2C)
                
                else:
                    ## remove KMT2C clone ids

                    pat_probs_no_KMT2C = []

                    pat_probs_test = pat_probs.loc[clone_idx][0]
                    pat_probs_no_KMT2C.append(pat_probs_test)
                    logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
                    running_score.extend(logs_no_KMT2C)
                
        score = np.sum(running_score)
        score_vector.append(score)
    

    return np.array(score_vector)

In [132]:
## my indicies are wrong :( I need to sort the clones the same way in the coding maf

In [133]:
kmt2c_Pm[kmt2c_Pm['patient']=='1326']

,clone,0,1,patient
54,1326_CL1,0.779492,0.220508,1326
55,1326_CL10,0.905479,0.094521,1326
56,1326_CL12,0.670312,0.329688,1326
57,1326_CL13,0.362515,0.637485,1326
58,1326_CL2,0.741343,0.258657,1326
59,1326_CL3,0.975526,0.024474,1326
60,1326_CL5,0.975526,0.024474,1326
61,1326_CL6,0.840239,0.159761,1326
62,1326_CL7,0.928278,0.071722,1326
63,1326_CL8,0.975526,0.024474,1326


In [134]:
obs_scores =calculate_scores(mut_clone_indices_obs)

In [144]:
mut_clone_indices_obs

{0: [],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [58, 64],
 6: [],
 7: [73, 74],
 8: [80, 81],
 9: [],
 10: [],
 11: [109, 114, 115, 117, 118, 120],
 12: [125, 128, 131, 133],
 13: [],
 14: []}

In [135]:
obs_scores

array([ 2.56378182,  2.74361215,  2.78723967,  2.83592867,  2.74375897,
        4.54779441,  2.89196222,  2.80316138,  3.40383483,  2.78576736,
        2.76256343, 14.71403037,  8.82017986,  2.76006954,  2.67642353])

In [69]:
CG.shape

(159, 6790)

In [70]:
CG[:, subset_idx_mat].shape

(159, 1)

In [210]:
def compute_pat_sums(pat_idx, mat):
    return (npg.aggregate(pat_idx, mat, axis = 0) > 0).sum(0)
# observed not null, really
null_pat_sum = compute_pat_sums(c2pat, CG[:, subset_idx_mat])

In [251]:
## this gets me a list of patients with a 0 or 1 depending on whether a given patient was mutated more than once in the target gene

def compute_clone_sums(pat_idx, mat):
    return (npg.aggregate(pat_idx, mat, axis = 0) > 1).sum(1)

clone_per_patient = compute_clone_sums(c2pat, CG[:, subset_idx_mat])


In [252]:
clone_per_patient

array([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0])

In [267]:
(npg.aggregate(c2pat, CG[:, subset_idx_mat], axis = 0))

array([[0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [6],
       [4],
       [0],
       [0]], dtype=int16)

In [269]:
## this gets # of clones per patient that are mutated
def compute_num_mut_clones(pat_idx, mat):
    return (npg.aggregate(c2pat, CG[:, subset_idx_mat], axis = 0))
    
compute_num_mut_clones(c2pat, CG[:, subset_idx_mat])



array([[0],
       [0],
       [0],
       [0],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [6],
       [4],
       [0],
       [0]], dtype=int16)

In [71]:
gene_patient_matrix = np.zeros_like(CG)
for i, pat in enumerate(c2pat):
    gene_patient_matrix[pat, :] |= CG[i, :] 
n_mut_pats = gene_patient_matrix.sum(axis=0)
print(n_mut_pats)

[2 2 1 ... 1 3 1]


In [93]:
obs_scores

[2.5637818168639304,
 2.7436121458661056,
 2.787239669333773,
 2.8359286696937476,
 2.7437589710934858,
 4.547794414031646,
 2.891962217736308,
 2.803161382745473,
 3.4038348338438515,
 2.7857673611956955,
 2.7625634283606044,
 14.714030365894862,
 8.820179860729802,
 2.7600695394391015,
 2.6764235305563515]

In [126]:
len(np.zeros_like( obs_scores))

15

In [136]:
 obs_scores.shape

(15,)

In [145]:
all_patients = list(coding_maf['Patient_ID'].unique())
# precompute nonzero elements
hp = curveball.find_presences(CG)

#
# run permutations (in parallel)
pool = multiprocessing.Pool()

n_perm = 10000
n_chunk = 1000

n_genes=1
n_iterations = n_perm // n_chunk 
total_clones = np.arange(1, len(all_patients) )
mutation_counts = np.zeros((n_genes, len(all_patients)))
test_matrix = np.zeros((n_iterations, len(all_patients)))

test = np.zeros((len(obs_scores), 1))
print(test.shape)
for p_chunk in range(n_perm//n_chunk):
    permute_futures = [None]*n_chunk
    for p in range(n_chunk):
        
        permute_futures[p] = pool.apply_async(curveball.curve_ball, (CG, hp, 1000) )
    # compute permutation p-values
    for p in range(n_chunk):

        # one iteration will return a shuffled distribution of clones in patients
       
        itr_test = ( compute_mut_clone_indices(c2pat, permute_futures[p].get()[:, subset_idx_mat]))

        ## then I compute the score of for the given iteration and take the test
        scores_per_itr = calculate_scores(itr_test)
        
        # test should be have a row for every patient
        #test += (  calculate_scores(compute_mut_clone_indices(c2pat, permute_futures[p].get()[:, subset_idx_mat])) <= obs_scores )
        test += (scores_per_itr >= obs_scores).reshape(len(obs_scores), 1)
    
    # tracking the final deviation from the observed distribution of # patients with a given mutation on each step         
    test_matrix[p_chunk, :] = scores_per_itr.copy()
    
    print(f"Chunk {p_chunk + 1}/{n_perm//n_chunk} finished.")


p_values = test / (n_chunk*(n_perm//n_chunk))
#Cb = pd.DataFrame({ "gene" : genes[subset_idx_mat], "p" : test/(n_chunk*(n_perm//n_chunk)) })

"""
Cb = pd.DataFrame({ "gene" : genes[subset_idx_mat], "p" : test/(n_chunk*(n_perm//n_chunk)),
                      "n_mut_clones": num_total_mutations[subset_idx_mat], 
                    "n_mut_pats": n_mut_pats[subset_idx_mat] })
"""

(15, 1)
Chunk 1/10 finished.
Chunk 2/10 finished.
Chunk 3/10 finished.
Chunk 4/10 finished.
Chunk 5/10 finished.
Chunk 6/10 finished.
Chunk 7/10 finished.
Chunk 8/10 finished.
Chunk 9/10 finished.
Chunk 10/10 finished.


'\nCb = pd.DataFrame({ "gene" : genes[subset_idx_mat], "p" : test/(n_chunk*(n_perm//n_chunk)),\n                      "n_mut_clones": num_total_mutations[subset_idx_mat], \n                    "n_mut_pats": n_mut_pats[subset_idx_mat] })\n'

In [ ]:
## I want a P value for every patient actually

In [151]:
pd.DataFrame({'patient':list(unique_pats), 'CB_p_vals_KMT2C':list(float(i) for i in p_values)})

/var/folders/rx/pxwbtcqd6k38h9yc5d3m4k1w0000gp/T/ipykernel_20715/2260590307.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pd.DataFrame({'patient':list(unique_pats), 'CB_p_vals_KMT2C':list(float(i) for i in p_values)})


,patient,CB_p_vals_KMT2C
0,1002,0.9650
1,1035,1.0000
2,1045,0.9854
3,1078,0.9923
4,1113,1.0000
5,1326,0.0146
6,1534,0.9956
7,1547,0.2132
8,1558,0.0704
9,1598,0.9938


In [146]:
p_values

array([[0.965 ],
       [1.    ],
       [0.9854],
       [0.9923],
       [1.    ],
       [0.0146],
       [0.9956],
       [0.2132],
       [0.0704],
       [0.9938],
       [0.9663],
       [0.0122],
       [0.3852],
       [0.9703],
       [0.9881]])

In [139]:
len(test)

15

In [140]:
itr_test

{0: [3, 5, 7, 9, 10, 12],
 1: [],
 2: [],
 3: [],
 4: [],
 5: [64],
 6: [],
 7: [69, 70],
 8: [],
 9: [92, 97],
 10: [],
 11: [109, 112],
 12: [125, 129, 130],
 13: [],
 14: []}

In [141]:
test_matrix.T[0]

array([ 8.2451863 , 10.58791566,  4.47525328,  2.56378182,  8.48429986,
       11.39357026,  9.05015   ,  2.56378182,  3.82436107, 15.90810085])

In [142]:
transpose_test[11]

array([ 5.22393907,  3.95337934,  6.88312278, 10.81665645,  4.88162259,
       14.0131802 ,  8.00305622,  4.51553664,  7.97158779,  8.20599173])

In [143]:
obs_scores[0]

2.5637818168639304